In [ ]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams["figure.figsize"] = [10, 5]

In [ ]:
df = pd.read_csv('chordify_with_rn_200_1991-01-05_to_2022-10-01.csv', parse_dates=['date'])
df['year'] = df['date'].dt.year # Add year

# 'roman_numerals' column is a JSON string. Turn the values into lists.
df.roman_numerals = df.roman_numerals.apply(json.loads)
print('Total songs: ', len(df))

# Remove any `None` values (which is the value when no RN could be found for the chord).
df = df[df.roman_numerals.apply(lambda x: None not in x)]
print('Songs with all roman numerals resolved (used for analysis): ', len(df))

# Create a copy of the row-per-song dataframe for later.
song_df = df

# Create our final main dataframe with one row per-chord-per-song.
df = df.explode('roman_numerals')

In [ ]:
# Check some summary stats.
print('Total number of songs: {:,}'.format(len(df.groupby(['artist', 'song']).count())))
print('Total number of chord instances: {:,}'.format(len(df)))
print('Average chord instances per song: {:.3f}'.format(df.groupby(['artist', 'song']).roman_numerals.count().mean()))
print('Average unique chords per song: {:.3f}'.format(df.groupby(['artist', 'song']).roman_numerals.nunique().mean()))
# Show all unique roman numeral chords in our dataset.
print('\n{} unique chords:'.format(len(df.roman_numerals.unique())))
print(', '.join(sorted(df.roman_numerals.unique())))

In [ ]:
df.groupby(['artist', 'song']).roman_numerals.nunique().hist(bins=range(0, 18, 2))
plt.title('Distribution of unique chord count')
plt.xlabel('Number of unique chords')
_ = plt.ylabel('Number of songs')

In [ ]:
# Histogram of RN counts across the full dataset:
_ = df.roman_numerals.value_counts().plot.bar(title='Chord counts by type', width=0.9)

In [ ]:
# Same, but only the top 10 most popular chords:
_ = df.roman_numerals.value_counts().nlargest(10).plot.bar(title='Chord counts by type (Top 10)', width=0.9)

In [ ]:
df.groupby('year').roman_numerals.value_counts().groupby('year').head(3).unstack()\
    .plot.bar(stacked=True, title='Three most popular chords in each year', ylabel='Chord count', xlabel='')
_ = plt.legend(bbox_to_anchor=(1, 1)) # Move legend outside plot

In [ ]:
song_df['rn_quadgrams'] = song_df.roman_numerals.apply(
    lambda items, n=4: [items[i:i+n] for i in range(len(items) - n + 1)]
)

In [ ]:
QUADGRAM_1564 = ['I', 'V', 'vi', 'IV']
QUADGRAM_1564_ROTATIONS = [list(np.roll(QUADGRAM_1564, rotation)) for rotation in range(4)]

song_df['is_1564'] = song_df.rn_quadgrams.apply(
    lambda quadgrams: [quadgram in QUADGRAM_1564_ROTATIONS for quadgram in quadgrams]
)

song_df['has_1564'] = song_df.is_1564.apply(any)
song_df['num_1564'] = song_df.is_1564.apply(lambda is_1564: is_1564.count(True))

In [ ]:
_ = song_df.groupby('year').has_1564.value_counts().unstack()\
    .plot.bar(stacked=True, title='Number of songs per year with/without a 1564 quadgram', xlabel='', ylabel='Songs')

In [ ]:
ndf = song_df.groupby('year').has_1564.value_counts().unstack()
ndf

In [ ]:
rdf = pd.DataFrame()
rdf['year'] = range(1991,2023)
rdf

In [ ]:
rdf['total_songs'] = 200
rdf

In [ ]:
rdf['with_the_progression'] = ndf.loc[:, True].tolist()
rdf

In [ ]:
rdf['ratio'] = rdf['with_the_progression'] / rdf['total_songs']
rdf

In [ ]:
plt.rc('axes', labelcolor='#474e68')
plt.rc('text', color='#474e68')
plt.rc('xtick', color='#474e68')
plt.rc('ytick', color='#474e68')
plt.rcParams.update({'font.size': 8})
plt.rc('font', weight='bold')
chart = rdf.plot.bar(title = 'Songs with and without The Progression per Year',
    ylabel = 'total number of songs per year',
    x = 'year',
    y = ['with_the_progression', 'total_songs'],
    color={'year': '#a7d2cb', 'ratio': '#f2d388', 'with_the_progression': '#c98474', 'total_songs': '#874c62'},
    stacked=False,
    rot=90)
chart2 = rdf.plot(secondary_y=True,
                  y='ratio',
                  color='#a7d2cb',
                  marker='o',
                  ax=chart,
                  rot=90)
chart2.set_ylabel('ratio')

In [ ]:
ax = song_df.groupby('year').num_1564.sum()\
    .plot.bar(title='Total number of I-V-vi-IV quadgrams per year', xlabel='', ylabel='I-V-vi-IV quadgrams', grid=True)
ax.set_axisbelow(True)

In [ ]:
year_ratios = song_df.groupby('year').apply(lambda sub: sub.num_1564.sum() / sub.rn_quadgrams.apply(len).sum())
ax = year_ratios\
    .plot.bar(title='Ratio of I-V-vi-IV quadgrams per year', xlabel='', ylabel='I-V-vi-IV quadgram ratio', grid=True)
ax.set_axisbelow(True)

## Mann-Kendall Tests

In [ ]:
!pip install pymannkendall

In [ ]:
import pymannkendall as mk

In [ ]:
trend_years = year_ratios.index.values[:-1]
trend_df = pd.DataFrame([mk.original_test(year_ratios.loc[year:]) for year in trend_years], index=trend_years)
trend_df.head() # Each row has all values returned from the Mann-Kendall Trend Test from the row year to 2022.

In [ ]:
ax = trend_df.p.plot(
    title='Each vertical bar shows estimated trend to 2022. Lower $p$ value is more confident.',
    ylabel='$p$ value', xlim=(1991, 2021), ylim=(0,1),  lw=3, label='$p$ value', marker='o'
)
h = ax.hlines(0.05, xmin=1991, xmax=2021, linestyles='dashed', color='k', label='$p = 0.05$')

plt.suptitle('Mann-Kendall trend test for each year', fontsize=16)
colors = {'decreasing': 'red', 'no trend': 'gray', 'increasing': 'green'}
labelled_trends = set() # Keep track of which trend backgrounds we've labelled, to avoid duplicate legend items.
for year, trend in trend_df.trend.items():
    ax.axvspan(year - 0.5, year + 0.5, alpha=0.35, color=colors[trend],
               label='' if trend in labelled_trends else trend.capitalize())
    labelled_trends.add(trend)
_ = ax.legend(loc='upper left'  )

## Per song proportion

In [ ]:
song_df['num_quadgram'] = song_df.rn_quadgrams.apply(lambda x: len(x))
song_df['1564_ratio'] = song_df['num_1564'] / song_df['num_quadgram']
song_df

In [ ]:
import seaborn as sns

ax = sns.stripplot(data=song_df, x='year', y='1564_ratio', s=2 ,jitter=0.2)
ax.tick_params(axis='x', rotation=90)

In [ ]:
import statsmodels.api as sm

ols_trend = []
for exp_year in trend_years:
    X = song_df.loc[song_df['year']>=exp_year]['year'].tolist()
    Y = np.array(song_df.loc[song_df['year']>=exp_year]['1564_ratio'].tolist())
    X = sm.add_constant(X)
    est = sm.OLS(Y, X, missing='drop')
    est_fit = est.fit()
    trend = 'No Trend'
    if est_fit.f_pvalue < 0.05:
        if est_fit.params[1] > 0:
            trend = 'Increasing'
        else:
            trend = 'Decreasing'
    ols_trend.append({'year': exp_year,
                      'p_value': est_fit.f_pvalue,
                      'R_squared': est_fit.rsquared,
                      'intercept': est_fit.params[0],
                      'slope': est_fit.params[1],
                      'trend': trend})
ols_trend = pd.DataFrame(ols_trend)

In [ ]:
ols_trend

In [ ]:
ax = ols_trend.plot(
    title='Each vertical bar shows estimated trend to 2022. Lower $p$ value is more confident.',
    x='year', y='p_value',
    ylabel='$p$ value', xlim=(1991, 2021), ylim=(0,1),  lw=3, label='$p$ value', marker='o'
)
h = ax.hlines(0.05, xmin=1991, xmax=2021, linestyles='dashed', color='k', label='$p = 0.05$')

plt.suptitle('OLS regression and their F-test p values for each year', fontsize=16)
colors = {'Decreasing': 'red', 'No Trend': 'gray', 'Increasing': 'green'}
labelled_trends = set() # Keep track of which trend backgrounds we've labelled, to avoid duplicate legend items.
for year, trend in zip(ols_trend.year.to_list(), ols_trend.trend.to_list()):
    ax.axvspan(year - 0.5, year + 0.5, alpha=0.35, color=colors[trend],
               label='' if trend in labelled_trends else trend.capitalize())
    labelled_trends.add(trend)
_ = ax.legend(loc='upper left'  )